In [11]:
import pandas as pd
import os


def consolidar_archivo_cmf(ruta_archivo, categoria_principal):
    """
    Lee un archivo Excel de la CMF, extrae metadatos (subtipo, moneda)
    y despivota las series de tiempo de cada hoja válida.
    """
    print(f"Procesando: {categoria_principal} -> {ruta_archivo}")

    if not os.path.exists(ruta_archivo):
        print(f"  [!] Error: No se encontró el archivo {ruta_archivo}")
        return pd.DataFrame()

    xls = pd.ExcelFile(ruta_archivo)
    hojas_metadatos = ['inicio', 'índice general', 'notas generales', 'glosario', 'índice_general', 'notas_generales']

    lista_dataframes = []

    for hoja in xls.sheet_names:
        # 1. Omitir hojas que no son de datos
        if any(ignorar in hoja.lower() for ignorar in hojas_metadatos):
            continue

        try:
            # 2. Extraer Metadatos
            df_meta = pd.read_excel(xls, sheet_name=hoja, nrows=5, header=None)

            subtipo = str(df_meta.iloc[2, 1]).strip()

            moneda_str = str(df_meta.iloc[3, 1]).strip().lower()
            moneda = 'USD' if 'dólares' in moneda_str else 'CLP'

            # 3. Extraer Tabla de Datos
            df_datos = pd.read_excel(xls, sheet_name=hoja, skiprows=5)

            # Identificar la columna del Banco
            col_banco = df_datos.columns[1]
            df_datos = df_datos.rename(columns={col_banco: 'Institución Financiera'})

            df_datos = df_datos.drop(columns=[df_datos.columns[0]])

            # 4. Limpieza de Datos

            # Eliminar filas donde el Banco sea nulo
            df_datos = df_datos.dropna(subset=['Institución Financiera'])
            df_datos['Institución Financiera'] = df_datos['Institución Financiera'].astype(str).str.strip().str.upper()

            # Filtrar filas que son sumatorias o texto de pie de página
            palabras_excluir = ['TOTAL', 'SISTEMA', 'FUENTE', 'NAN']
            df_datos = df_datos[~df_datos['Institución Financiera'].isin(palabras_excluir)]
            df_datos = df_datos[~df_datos['Institución Financiera'].str.contains('TOTAL', na=False)]

            # 5. Data Wrangling
            df_melted = df_datos.melt(id_vars=['Institución Financiera'], var_name='Fecha', value_name='Valor')

            # Eliminar registros sin monto
            df_melted = df_melted.dropna(subset=['Valor'])

            # Enriquecer con los metadatos y la categoría
            df_melted['Tipo'] = categoria_principal
            df_melted['Subtipo'] = subtipo
            df_melted['Tipo de Moneda'] = moneda

            lista_dataframes.append(df_melted)

        except Exception as e:
            print(f"  [!] Advertencia: No se pudo procesar la hoja {hoja}. Detalle: {e}")

    # Unir todos los dataframes de las hojas procesadas
    if lista_dataframes:
        return pd.concat(lista_dataframes, ignore_index=True)
    else:
        return pd.DataFrame()


# EJECUCIÓN DEL PROCESO

# Definir rutas de los archivos
archivos = {
    'Colocaciones': '/content/sdbcol_092021 (1).xlsx',
    'Depósitos': '/content/sdbdep_092021 (1).xlsx',
    'Inversiones': '/content/sdbinv_092021 (1).xlsx'
}

dataframes_finales = []

# Procesar cada archivo individualmente
for categoria, ruta in archivos.items():
    df_resultado = consolidar_archivo_cmf(ruta, categoria)
    if not df_resultado.empty:
        dataframes_finales.append(df_resultado)

# Consolidar todo en un único archivo maestro
if dataframes_finales:
    df_consolidado = pd.concat(dataframes_finales, ignore_index=True)

    # Asegurar que la fecha sea tipo datetime para facilitar análisis
    df_consolidado['Fecha'] = pd.to_datetime(df_consolidado['Fecha'], errors='coerce')

    # Exportar el resultado a CSV
    ruta_salida = '/content/BDC_Consolidado_Competencia.csv'
    df_consolidado.to_csv(ruta_salida, index=False, encoding='utf-8')

    print("\n" + "="*50)
    print("🚀 ¡Proceso de Data Wrangling completado con éxito!")
    print(f"Filas totales: {len(df_consolidado)}")
    print(f"Columnas generadas: {list(df_consolidado.columns)}")
    print(f"Archivo guardado en: {ruta_salida}")
    print("="*50)

    # Imprimir una muestra de cómo quedó estructurado
    display(df_consolidado.head())
else:
    print("\n[!] No se generó información. Verifica que las rutas de los archivos excel sean correctas.")

Procesando: Colocaciones -> /content/sdbcol_092021 (1).xlsx
Procesando: Depósitos -> /content/sdbdep_092021 (1).xlsx
Procesando: Inversiones -> /content/sdbinv_092021 (1).xlsx

🚀 ¡Proceso de Data Wrangling completado con éxito!
Filas totales: 137455
Columnas generadas: ['Institución Financiera', 'Fecha', 'Valor', 'Tipo', 'Subtipo', 'Tipo de Moneda']
Archivo guardado en: /content/BDC_Consolidado_Competencia.csv


,Institución Financiera,Fecha,Valor,Tipo,Subtipo,Tipo de Moneda
0,BANCO BICE,2008-01-31,1763607.105975,Colocaciones,Colocaciones totales por tipo de crédito,CLP
1,BANCO BTG PACTUAL CHILE (9),2008-01-31,ND,Colocaciones,Colocaciones totales por tipo de crédito,CLP
2,BANCO CONSORCIO (1),2008-01-31,33929.700249,Colocaciones,Colocaciones totales por tipo de crédito,CLP
3,BANCO DE CHILE (2),2008-01-31,12877155.584751,Colocaciones,Colocaciones totales por tipo de crédito,CLP
4,BANCO DE CRÉDITO E INVERSIONES,2008-01-31,7808179.921376,Colocaciones,Colocaciones totales por tipo de crédito,CLP
